# SKU Prognoz

In [ ]:
%%time
import pandas as pd
from neuralprophet import NeuralProphet

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/SKU for prophet.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели NeuralProphet
data = data.rename(columns={'Дата': 'ds', 'Код товара': 'SKU', 'Продажи, кг': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    # Проверка на наличие пропущенных значений в столбце 'y'
    if sku_data['y'].notnull().sum() < 2:
        continue  # Пропускаем прогноз для товаров с меньше чем 2 непропущенными значениями 'y'

    # Создание и обучение модели NeuralProphet для каждого товара
    model = NeuralProphet(
    yearly_seasonality=10,  # Больше гармоник для более гибкой сезонности
    weekly_seasonality=True,
    growth="logistic",      # Нелинейный тренд
    learning_rate=0.01,     # Более медленный градиентный спуск
    epochs=250              # Больше эпох для более детальной тренировки
    )

    model.fit(sku_data[['ds', 'y']], freq='D')

    # Создание DataFrame для прогноза на N дней вперед
    future = model.make_future_dataframe(sku_data[['ds', 'y']], periods=450)     #Период
    forecast = model.predict(future)
    forecast['SKU'] = sku  # Добавление информации о товаре

    # Замена отрицательных значений прогноза на 0
    forecast['yhat1'] = forecast['yhat1'].where(forecast['yhat1'] > 0, 0)

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
selected_columns = ['ds', 'SKU', 'yhat1']
all_forecasts[selected_columns].to_excel('прогноз_продаж_по_SKU_neuralprophet.xlsx', index=False)

# Гибридный NeuralProphet+CatBoost

In [ ]:
import pandas as pd
from neuralprophet import NeuralProphet
from catboost import CatBoostRegressor

# Загрузка данных
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/SKU for prophet.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных
data = data.rename(columns={'Дата': 'ds', 'Код товара': 'SKU', 'Продажи, кг': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по SKU
grouped_data = data.groupby('SKU')

# DataFrame для хранения прогнозов NeuralProphet на будущее
final_forecast = pd.DataFrame()

# Прогнозирование NeuralProphet для каждого SKU
for sku, sku_data in grouped_data:
    if sku_data['y'].notnull().sum() < 2:
        continue

    # Создание и обучение модели NeuralProphet
    model_np = NeuralProphet(
        yearly_seasonality=10,
        weekly_seasonality=True,
        growth="logistic",
        learning_rate=0.01,
        epochs=250
    )
    model_np.fit(sku_data[['ds', 'y']], freq='D')

    # Прогноз на будущее на 80 дней вперед
    future = model_np.make_future_dataframe(df=sku_data[['ds', 'y']], periods=80)       #Период      
    forecast_np = model_np.predict(future)       
    forecast_np['yhat1'] = forecast_np['yhat1'].where(forecast_np['yhat1'] > 0, 0)
    forecast_np['SKU'] = sku

    # Отбор прогнозов только на будущее
    forecast_np_future = forecast_np[forecast_np['ds'] > sku_data['ds'].max()]

    # Добавление прогноза в финальный DataFrame
    final_forecast = pd.concat([final_forecast, forecast_np_future[['ds', 'SKU', 'yhat1']]])

# Создание признаков для CatBoost
data['month'] = data['ds'].dt.month
data['dayofweek'] = data['ds'].dt.dayofweek

# Инициализация и обучение CatBoost
X = data[['month', 'dayofweek', 'SKU']]
y = data['y']

model_catboost = CatBoostRegressor(iterations=500, learning_rate=0.05, depth=8, cat_features=['SKU', 'month', 'dayofweek'], verbose=100)
model_catboost.fit(X, y)

# Прогнозирование на будущее с использованием CatBoost
final_forecast['month'] = final_forecast['ds'].dt.month
final_forecast['dayofweek'] = final_forecast['ds'].dt.dayofweek

X_future = final_forecast[['month', 'dayofweek', 'SKU']]
final_forecast['forecast'] = model_catboost.predict(X_future)

# Оставляем только нужные столбцы и сохраняем результат
final_forecast = final_forecast[['ds', 'SKU', 'forecast']]
final_forecast.to_excel('гибридный_прогноз_neuralprophet_catboost_final.xlsx', index=False)

print("Итоговый прогноз на 80 дней вперед сохранен в 'гибридный_прогноз_neuralprophet_catboost_final.xlsx'")

ERROR - (NP.config.__post_init__) - Invalid trend growth 'logistic'. Set to 'linear'
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 75.0% of the data.
WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.w

Training: |                                                                                      | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\pytorch_lightning\utilities\data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\pytorch_lightning\utilities\data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 12. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_freque

Predicting: |                                                                                    | 0/? [00:00<…

...

WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.847% of the data.
WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use

Predicting: |                                                                                    | 0/? [00:00<…

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
ERROR - (NP.config.__post_init__) - Invalid trend growth 'logistic'. Set to 'linear'
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.847% of the data.
WARNING - (py.warnings._showwarnmsg) - C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  

Training: |                                                                                      | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…